In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

if False: # if True: turn off GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
    gpu = False
else:
    gpu = True

from tensorflow import keras
from copy import deepcopy
import tensorflow as tf
from glob import glob
import numpy as np
import random


from utils.cityscapesSequence import CitySequence, labels
import utils.dataUtils as dataUtils
from modelclass import DeeplabV3

if gpu:
    gpus = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=(1024 * 8))])

print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.10.1


In [2]:
DATA_DIR = os.path.join("..", "cityscapes", "allData")

batchSize = 8
imageSize = 256
lrs = [5e-4, 1e-4, 5e-5, 1e-5, 5e-6]
alphas = [1., 0.9, 0.8, 0.7, 0.6]


##### Dataset


In [ ]:
def getData(path="", type="gtFine", extra=False, nSplits=5, seed=0):
    # Load data
    xTrainVal, yTrainVal, xTest, yTest = dataUtils.getXY(path=path, extra=extra, type=type)
    print("TrainVal: ", len(xTrainVal), "Test: ", len(yTest))

    # Split the data in train and validation folds
    return dataUtils.splitTrainValFolds(xTrainVal, yTrainVal, nSplits=nSplits, seed=seed)


def createDatasetObject(trainValFolds, imageSize=imageSize, batchSize=1, remap="binary"):
    for i, (xTrain, yTrain, xVal, yVal) in enumerate(trainValFolds):
        print("Fold", i, "Train: ", len(xTrain), "Val: ", len(xVal))

        dataUtils.dataShuffle(xData=xTrain, yData=yTrain)

        # Create the dataset
        trainDataset = CitySequence(xTrain, yTrain, batchSize=batchSize, imageSize=imageSize, remap=remap, blur=0, crop=True, horizontalFlip=True, verticalFlip=True, brightness=0.1)
        valDataset = CitySequence(xVal, yVal, batchSize=batchSize, imageSize=imageSize, remap=remap, blur=0, crop=True, horizontalFlip=False, verticalFlip=False, brightness=0)

        yield trainDataset, valDataset, trainDataset.nClasses


kf = getData(path=DATA_DIR, type="gtFine", extra=False, nSplits=5, seed=13)
data = createDatasetObject(kf, imageSize=imageSize, batchSize=batchSize, remap="binary")

##### Train

In [4]:
losses = {}

for i, (trainDataset, valDataset, nClasses) in enumerate(data):
    for lr in lrs:
        for alpha in alphas:
            print("Fold:", i, "learning rate:", lr, "alpha:", alpha)
            
            MODEL_NAME = "deeplabv3plus_fold_" + str(i) + "_lr_" + str(lr) + "_alpha_" + str(alpha) + "_binary" + str(imageSize)

            model = DeeplabV3(imageSize=imageSize, nClasses=nClasses, alpha=alpha, modelName=MODEL_NAME)
            
            model.run(trainDataset, valDataset, epochs=1000, learningRate=lr, batchSize=batchSize, monitor="val_loss", log=True)

            # Evaluate the model
            losses[MODEL_NAME] = model.evaluate(valDataset)
            print("losses", losses)

Fold 0 Train:  2780 Val:  695
Fold: 0 learning rate: 0.0005 alpha: 1.0
Epoch 1/1000
348/348 [==============================] - ETA: 0s - loss: 57.3811 - diceAccuracy: 0.9060 - jaccardDistance: 1.6425
Epoch 1: val_loss improved from inf to 8.45420, saving model to model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


348/348 [==============================] - 402s 1s/step - loss: 57.3811 - diceAccuracy: 0.9060 - jaccardDistance: 1.6425 - val_loss: 8.4542 - val_diceAccuracy: 0.5075 - val_jaccardDistance: 6.5977 - lr: 5.0000e-04
Epoch 2/1000
348/348 [==============================] - ETA: 0s - loss: 2.2428 - diceAccuracy: 0.9298 - jaccardDistance: 1.2837
Epoch 2: val_loss improved from 8.45420 to 0.47994, saving model to model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


...

348/348 [==============================] - 352s 1s/step - loss: 0.1628 - diceAccuracy: 0.9505 - jaccardDistance: 0.9308 - val_loss: 0.1649 - val_diceAccuracy: 0.9462 - val_jaccardDistance: 0.9955 - lr: 5.0000e-04
Epoch 220/1000
348/348 [==============================] - ETA: 0s - loss: 0.2968 - diceAccuracy: 0.9495 - jaccardDistance: 0.9471
Epoch 220: val_loss did not improve from 0.16490
348/348 [==============================] - 326s 936ms/step - loss: 0.2968 - diceAccuracy: 0.9495 - jaccardDistance: 0.9471 - val_loss: 0.3489 - val_diceAccuracy: 0.9437 - val_jaccardDistance: 1.0451 - lr: 5.0000e-04
Epoch 221/1000
348/348 [==============================] - ETA: 0s - loss: 0.1988 - diceAccuracy: 0.9500 - jaccardDistance: 0.9395
Epoch 221: val_loss did not improve from 0.16490
348/348 [==============================] - 328s 943ms/step - loss: 0.1988 - diceAccuracy: 0.9500 - jaccardDistance: 0.9395 - val_loss: 0.5048 - val_diceAccuracy: 0.4399 - val_jaccardDistance: 7.1520 - lr: 5.0000e-

INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


INFO:tensorflow:Assets written to: model-saved\deeplabv3plus_fold_0_lr_0.0005_alpha_1.0_binary256\assets


348/348 [==============================] - 352s 1s/step - loss: 0.1623 - diceAccuracy: 0.9506 - jaccardDistance: 0.9296 - val_loss: 0.1638 - val_diceAccuracy: 0.9467 - val_jaccardDistance: 0.9970 - lr: 5.0000e-04
Epoch 237/1000
348/348 [==============================] - ETA: 0s - loss: 0.1683 - diceAccuracy: 0.9512 - jaccardDistance: 0.9176
Epoch 237: val_loss did not improve from 0.16384
348/348 [==============================] - 339s 974ms/step - loss: 0.1683 - diceAccuracy: 0.9512 - jaccardDistance: 0.9176 - val_loss: 0.4007 - val_diceAccuracy: 0.5956 - val_jaccardDistance: 5.7122 - lr: 5.0000e-04
Epoch 238/1000
348/348 [==============================] - ETA: 0s - loss: 0.1638 - diceAccuracy: 0.9507 - jaccardDistance: 0.9275
Epoch 238: val_loss did not improve from 0.16384
348/348 [==============================] - 352s 1s/step - loss: 0.1638 - diceAccuracy: 0.9507 - jaccardDistance: 0.9275 - val_loss: 0.3264 - val_diceAccuracy: 0.7082 - val_jaccardDistance: 4.4758 - lr: 5.0000e-04


KeyboardInterrupt: 

In [ ]:
losses

{}

##### TFLITE Converter


In [ ]:
from utils.imageUtils import representativeDatasetGen

representativeData = representativeDatasetGen(path="../cityscapes/alldata")

In [ ]:
MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

import models
importlib.reload(models)

image_size = 256
num_classes = 2
alpha = 1

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)
model.load_weights(MODEL_PATH)

model.toTFlite(representative_dataset_gen=representativeData, supported_ops=[tf.lite.OpsSet.TFLITE_BUILTINS])

MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)
model.load_weights(MODEL_PATH)

model.toTFlite(representative_dataset_gen=representativeData)

In [ ]:
MODEL_NAME = "binary_rain_fog.hdf5"
MODEL_PATH = os.path.join("model-saved", MODEL_NAME)

import models
importlib.reload(models)

image_size = 256
num_classes = 2
alpha = 1

model = models.DeeplabV3(image_size, num_classes, alpha, modelName=MODEL_NAME)

model.load_weights(MODEL_PATH)

